In [1]:
from fit import *

fit(path='./resultDec_eef/',parameter='./eefParameters.npz',combined=0,visualize=1,outputpath='./')


Bad key "text.kerning_factor" on line 4 in
/home/rachel/miniconda3/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


result_eef1a1G4mut.mat
Tactive =  36554400.0 nnn =  0
estp1  0.03250940919674265
Fitting sum of two exponentials...


/home/rachel/Documents/Manuscript/script_fitLong/fitSumOf2ExponentialsMain.py:44: RuntimeWarning: overflow encountered in exp
  np.log((k[2]*np.exp(k[0]*xl)+ (1-k[2])*np.exp(k[1]*xl))/elong)/sNL*fact1,
/home/rachel/Documents/Manuscript/script_fitLong/fitSumOf2ExponentialsMain.py:44: RuntimeWarning: overflow encountered in true_divide
  np.log((k[2]*np.exp(k[0]*xl)+ (1-k[2])*np.exp(k[1]*xl))/elong)/sNL*fact1,
/home/rachel/Documents/Manuscript/script_fitLong/fitSumOf2ExponentialsMain.py:46: RuntimeWarning: overflow encountered in exp
  (k[2]*np.exp(k[0]*xl)+ (1-k[2])*np.exp(k[1]*xl)-elong)/sNL*fact2)) ### mixed
/home/rachel/Documents/Manuscript/script_fitLong/fitSumOf2ExponentialsMain.py:43: RuntimeWarning: invalid value encountered in log
  return np.concatenate((np.log((k[2]*np.exp(k[0]*xsg)+(1-k[2])*np.exp(k[1]*xsg))/eshort)/sNS*fact1,
/home/rachel/Documents/Manuscript/script_fitLong/fitSumOf2ExponentialsMain.py:44: RuntimeWarning: invalid value encountered in log
  np.log((k[2]*np.ex

Tactive =  31620060.0 nnn =  2
estp1  0.03739283449475071
Fitting sum of two exponentials...


/home/rachel/Documents/Manuscript/script_fitLong/fitSumOf2ExponentialsMain.py:44: RuntimeWarning: overflow encountered in exp
  np.log((k[2]*np.exp(k[0]*xl)+ (1-k[2])*np.exp(k[1]*xl))/elong)/sNL*fact1,
/home/rachel/Documents/Manuscript/script_fitLong/fitSumOf2ExponentialsMain.py:44: RuntimeWarning: overflow encountered in true_divide
  np.log((k[2]*np.exp(k[0]*xl)+ (1-k[2])*np.exp(k[1]*xl))/elong)/sNL*fact1,
/home/rachel/Documents/Manuscript/script_fitLong/fitSumOf2ExponentialsMain.py:46: RuntimeWarning: overflow encountered in exp
  (k[2]*np.exp(k[0]*xl)+ (1-k[2])*np.exp(k[1]*xl)-elong)/sNL*fact2)) ### mixed
/home/rachel/Documents/Manuscript/script_fitLong/fitSumOf2ExponentialsMain.py:43: RuntimeWarning: invalid value encountered in log
  return np.concatenate((np.log((k[2]*np.exp(k[0]*xsg)+(1-k[2])*np.exp(k[1]*xsg))/eshort)/sNS*fact1,
/home/rachel/Documents/Manuscript/script_fitLong/fitSumOf2ExponentialsMain.py:44: RuntimeWarning: invalid value encountered in log
  np.log((k[2]*np.ex

...

/home/rachel/Documents/Manuscript/script_fitLong/fit3.py:154: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  h.savefig(figfile)


trying least squares for mc = 0
trying least squares for mc = 1


/home/rachel/Documents/Manuscript/script_fitLong/fitSumOf3ExponentialsMain.py:46: RuntimeWarning: invalid value encountered in log
  ee = np.concatenate(((np.log(((k[3]*np.exp(k[0]*xsg)+k[4]*np.exp(k[1]*xsg)+(1-k[3]-k[4])*np.exp(k[2]*xsg))/eshort))/sNS*fact1,
/home/rachel/Documents/Manuscript/script_fitLong/fitSumOf3ExponentialsMain.py:47: RuntimeWarning: invalid value encountered in log
  np.log((k[3]*np.exp(k[0]*xl)+k[4]*np.exp(k[1]*xl)+(1-k[3]-k[4])*np.exp(k[2]*xl))/elong)/sNL*fact1,


trying least squares for mc = 2
trying least squares for mc = 3
trying least squares for mc = 4
trying least squares for mc = 5
trying least squares for mc = 6
trying least squares for mc = 7
trying least squares for mc = 8
trying least squares for mc = 9
trying least squares for mc = 10
trying least squares for mc = 11
trying least squares for mc = 12
trying least squares for mc = 13
trying least squares for mc = 14
trying least squares for mc = 15
trying least squares for mc = 16
trying least squares for mc = 17
trying least squares for mc = 18
trying least squares for mc = 19
trying least squares for mc = 20
trying least squares for mc = 21
trying least squares for mc = 22
trying least squares for mc = 23
trying least squares for mc = 24
trying least squares for mc = 25
trying least squares for mc = 26
trying least squares for mc = 27
trying least squares for mc = 28
trying least squares for mc = 29
trying least squares for mc = 30
trying least squares for mc = 31
trying least squar

/home/rachel/Documents/Manuscript/script_fitLong/fit3.py:154: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  h.savefig(figfile)
/home/rachel/Documents/Manuscript/script_fitLong/fit3.py:174: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  h.savefig(figfile)
No handles with labels found to put in legend.
/home/rachel/Documents/Manuscript/script_fitLong/fit3.py:191: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  h.savefig(figfile)
